In [1]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

d87b1f9d-8a0b-4a6e-83ab-881aab0fe457


In [2]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [ ]:
# payload-2 (Regression Alcohol dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "ALCOHOL_DATA_10L", 
"target_column": "QUALITY"}}'''.format(run_id)

In [3]:
# payload-3 (classification Employee dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.linear_model.LogisticRegression": null}}, 
"dataset": "EMPLOYEE_10L", 
"target_column": "LEAVEORNOT"}}'''.format(run_id)

In [2]:
# payload-4 (Regression Diamonds dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBRegressor": null}}, 
"dataset": "DIAMONDS", 
"target_column": "PRICE"}}'''.format(run_id)

In [3]:
exp_data

'{\n"source":"EXP", \n"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", \n"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", \n"run_id":"d87b1f9d-8a0b-4a6e-83ab-881aab0fe457", \n"exp_name": "Final_recipe", \n"algorithm_type":"classification", \n"algo_details": {"snowflake.ml.modeling.xgboost.XGBClassifier": null}, \n"dataset": "AIRLINE_DEP_DELAY_10K", \n"target_column": "DEP_DEL15"}'

In [4]:
from snowflake.snowpark.session import Session
# from snow_exp import run_experiment
from snow_exp import initiate_sproc_process, get_session
import json, asyncio, sys

In [9]:
#API function
def execute_sf_sproc(payload, sproc_name="run_experiment"):
    try:
        task = asyncio.create_task(initiate_sproc_process(payload))
#         sproc_response = initiate_sproc_process(payload)
#         print(sproc_response)
        print("Success")
    except Exception as ex:    
        return "Failed: ",ex

In [10]:
execute_sf_sproc(exp_data)

Success
Creating Snowflake Session object...
Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Session has been created !
Creating stored procedure...
Creating stored procedure...


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


Stored procedure has been created successfully!
Stored procedure has been created successfully!
Executing Procedure


In [ ]:
session = get_session("dummy","dummy")

In [6]:
sproc_res = session.call("run_experiment", exp_data)

SnowparkSQLException: (1304): 01b56c8e-0000-6fbd-0000-576d072d8fd6: 002140 (42601): SQL compilation error:
Unknown function RUN_EXPERIMENT

In [ ]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
exp_details=json.loads(exp_data)
session = get_session(exp_details.get("dataset"),exp_details.get("project_id"))
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.custom_package_usage_config['enabled'] = True
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", run_id)
session.query_tag=run_id

print("Executing Procedure")
procedure_response = session.call("run_experiment", exp_data)
# procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")